In [1]:
import numpy as np
import json


In [19]:
import json
dataset = 'bioasq'
input_file = f"/data/user_data/jhsia2/dbqa/data/{dataset}.jsonl"
dataset = dataset.split('_')[0]
output_file = f"/data/user_data/jhsia2/dbqa/data/gold-{dataset}.json"


is_bioasq = True if dataset == 'bioasq' else False
if is_bioasq:
    docid_key = 'pmid'
    docid_name = 'pm'
    section_key = 'section'
    section_name = 'sec'
else:
    docid_key = 'wikipedia_id'
    docid_name = 'wiki'
    section_key = 'start_paragraph_id'
    section_name = 'par'

# print('total num documents = 5903530')
num_par_docs = 0
gold_data_list = []
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for l_id, line in enumerate(infile):
        if (l_id%1000==0):
            print(l_id)
        data = json.loads(line)
        new_data = {}
        new_data['id'] = data['id']
        # print(data)
        new_data['input'] = data['input']
        new_data['output'] = {'answer_set' : set(), \
                              'title_set': set(), \
                              f'{docid_name}_id_set' : set(), \
                                f'{docid_name}_{section_name}_id_set': set()}
        
        for output in data['output']:
            ans = output.get('answer', None)
            if (ans):
                new_data['output']['answer_set'].add(ans)
            provs = output.get('provenance', None)
            if provs:
                for prov in provs:
                    wiki_id = prov.get(f'{docid_key}', None) 
                    if is_bioasq:
                        start_par_id = 0 if prov.get(f'{section_key}', None)== 'title' else 1
                        # end_par_id = prov.get('end_paragraph_id', None) 
                        end_par_id = start_par_id
                    else:
                        start_par_id = (int)(prov.get(f'{section_key}_id', None))
                        end_par_id = start_par_id
                    end_par_id = (int)(start_par_id)
                    title = prov.get('title', None)
                    if(wiki_id):
                        new_data['output'][f'{docid_name}_id_set'].add(wiki_id)
                    # if (start_par_id and end_par_id):
                        for p_id in range(start_par_id, end_par_id+1):
                            new_data['output'][f'{docid_name}_{section_name}_id_set'].add(wiki_id + '_' + str(p_id))
                    if title:
                        new_data['output']['title_set'].add(title)
        for k in new_data['output']:
            new_data['output'][k] = list(new_data['output'][k])
        # outfile.write(json.dumps(new_data) + "\n")
        gold_data_list.append(new_data)
        # break
print('done')
with open(output_file, 'w') as file:
    # Use json.dump to write the list of dictionaries to the file
    json.dump(gold_data_list, file, indent=4)

0
1000
2000
3000
done
